### **IMPORTAÇÕES**

In [1]:
import requests
import json

import pandas as pd
import numpy as np
import datetime

import plotly.express as px
import plotly.graph_objects as go

from tkinter import messagebox, ttk
import tkinter as tk

### **1ª ETAPA - REQUISIÇÃO**

In [2]:
from IPython.core.hooks import late_startup_hook
## Conexão com o Weather API

class Conexao_API():
  def __init__ (self, cidade):
    self.cidade = cidade
    self.key = "aa98651cbc2b13fb493344dbb5ae0fb1"
    self.geo = f"https://api.openweathermap.org/geo/1.0/direct?q={self.cidade}&appid={self.key}"
    self.coord = requests.get(self.geo).json()

  def iniciar(self):
    lat = round(self.coord[0]['lat'], 2)
    lon = round(self.coord[0]['lon'], 2)
    self.link = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={self.key}&units=metric&lang=pt_br"

    self.requisicao = requests.get(self.link)
    requisicao = self.requisicao.json()
    return requisicao

### **2ª ETAPA - PROCESSAMENTO**

In [69]:
## Tratamento de dados

class Database():
  def __init__(self, data, dados, grafico):
    self.data = data
    self.dados = dados.iniciar()
    self.grafico = grafico
  
  def database(self):

    # Criando dataframe dos dados de previsão
    self.COLUMNS = ['timestamp', 'temp_media', 'umidade', 'pressao', 'vel_vento', 'descricao']
    self.clima = pd.DataFrame(columns = self.COLUMNS)
    self.dias = self.dados['list']

    for i in range(len(self.dias)):
      self.clima.loc[len(self.clima)] = {
        self.COLUMNS[0]: pd.to_datetime(self.dias[i]['dt_txt']),
        self.COLUMNS[1]: self.dias[i]['main']['temp'],
        self.COLUMNS[2]: self.dias[i]['main']['humidity'],
        self.COLUMNS[3]: self.dias[i]['main']['pressure'],
        self.COLUMNS[4]: self.dias[i]['wind']['speed'],
        self.COLUMNS[5]: self.dias[i]['weather'][0]['description']}
      
    # Selecionando dados para o dia escolhido
    self.clima_dia = pd.DataFrame(columns = self.COLUMNS)

    for i in range(len(self.clima)):
      if str(self.clima['timestamp'][i])[:10] == self.data:
        self.clima_dia.loc[len(self.clima_dia)] = self.clima.loc[i]

    self.data = datetime.datetime.strptime(self.data, '%Y-%m-%d')
    self.str_data = str(self.data.day) + '-' + str(self.data.month) + '-' + str(self.data.year)

    # Relatório do tempo
    self.t_ymax = np.max(self.clima_dia['temp_media'])
    self.t_ymin = np.min(self.clima_dia['temp_media'])

    self.u_ymax = np.max(self.clima_dia['umidade'])
    self.u_ymin = np.min(self.clima_dia['umidade'])

    self.p_ymax = np.max(self.clima_dia['pressao'])
    self.p_ymin = np.min(self.clima_dia['pressao'])

    self.v_ymax = np.max(self.clima_dia['vel_vento'])
    self.v_ymin = np.min(self.clima_dia['vel_vento'])

    self.previsao = np.max(self.clima_dia['descricao'])

    print("RELATÓRIO DE PREVISÃO DO TEMPO " + self.str_data + ":\n")
    print("Temperatura:\nMáxima %.1f ºC\nMínima %.1f ºC\n" % (self.t_ymax, self.t_ymin))
    print("Umidade:\nMáxima %.1f %%\nMínima %.1f %%\n" % (self.t_ymax, self.t_ymin))
    print("Pressão:\nMáxima %.1f hPa\nMínima %.1f hPa\n" % (self.t_ymax, self.t_ymin))
    print("Velocidade do Vento:\nMáxima %.1f m/s\nMínima %.1f m/s\n" % (self.t_ymax, self.t_ymin))
    print("O tempo será predominantemente de: "+self.previsao)

  def graficos(self):

    # Criando figura
    fig = go.Figure()

    # Plotando os gráficos
    fig.add_trace(
      go.Scatter(
      x=list(self.clima_dia['timestamp']),
      y=list(self.clima_dia['temp_media']),
      name="Temperatura",
      visible=False,
      line=dict(
        color="#33CFA5")))
    
    fig.add_trace(
      go.Scatter(
      x=list(self.clima_dia['timestamp']),
      y=list(self.clima_dia['umidade']),
      name="Umidade",
      visible=False,
      line=dict(color="#F06A6A")))

    fig.add_trace(
      go.Scatter(
      x=list(self.clima_dia['timestamp']),
      y=list(self.clima_dia['pressao']),
      name="Pressão",
      visible=False,
      line=dict(color="#1260CC")))

    fig.add_trace(
      go.Scatter(
      x=list(self.clima_dia['timestamp']),
      y=list(self.clima_dia['vel_vento']),
      name="Vento",
      visible=False,
      line=dict(color="#993399")))
    
    ### TODO 3: Criar alertas com descrição ? (IMPORTANTE!)

    # Botões para cada gráfico

    fig.update_layout(
      title_text="Gráfico Previsão do Tempo " + self.str_data,
      xaxis_domain=[0.05, 1.0],
      updatemenus=[
        dict(
      type="buttons",
      direction="right",
      active=0,
      x=1.0,
      y=1.2,
      buttons=list([
        dict(label="Temperatura",
             method="update",
             args=[{"visible": [True, False, False, False]}]),
        dict(label="Umidade",
             method="update",
             args=[{"visible": [False, True, False, False]}]),
        dict(label="Pressão",
             method="update",
             args=[{"visible": [False, False, True, False]}]),
        dict(label="Vento",
             method="update",
             args=[{"visible": [False, False, False, True]}]),
             ]),
             )
             ])
    
    if self.grafico==1:
      fig.show()
    else:
      pass

### **3ª ETAPA - RESPOSTA**

In [70]:
class Janela():
    def __init__(self):
        self.cidade = ""
        self.data = ""
        self.cb_state = None

        ### Ação ao clicar no botão OK
        def ok_button():
            self.cidade = cidade_textbox.get()
            self.data = dias_dropdown.get()
            self.cb_state = check.get()

            ### Conferindo se todas as informações foram preenchidas
            if self.cidade and self.data:
                main_window.destroy()
            else:
                messagebox.showwarning("Atenção", "Complete os campos em branco")
            

        ### Iniciando a janela do usuário
        main_window = tk.Tk()
        main_window.geometry("400x200")

        ### Entrada da cidade
        main_window.title("Previsão do Tempo")
        cidade_label = tk.Label(main_window, text="Digite a cidade:")
        cidade_label.pack(anchor=tk.W,padx=10)
        cidade_textbox = tk.Entry(main_window)
        cidade_textbox.pack(anchor=tk.W,padx=10)

        ### Lista dos próximos 5 dias da semana
        hoje = datetime.datetime.today()
        semana = [(hoje + datetime.timedelta(days=x)).strftime("%Y-%m-%d") for x in range(5)]

        ### Entrada da data
        dias_label = tk.Label(main_window, text="Selecione a data:")
        dias_label.pack(anchor=tk.W, padx=10)
        dias_dropdown = ttk.Combobox(main_window, values=semana)
        dias_dropdown.pack(anchor=tk.W, padx=10)

        ### Checkbox do gráfico
        check = tk.BooleanVar()
        check_button = ttk.Checkbutton(main_window, text="Gráfico", variable=check, onvalue=True, offvalue=False)
        check_button.pack()
        # self.cb_state = check.get()

        submit_button = tk.Button(main_window, text='Ok', command=ok_button)
        submit_button.pack(anchor=tk.W,padx=10,pady=10)
        main_window.mainloop()

    def retorno(self):
        return [self.cidade, self.data, self.cb_state]

### Salvando as informações de cidade de data inseridas pelo usuário na janela

cidade, data, grafico = Janela().retorno()

### Inicialização da API e Busca pela cidade

dados = Conexao_API(cidade)

### Seleção do período de tempo a ser consultado

database = Database(data, dados, grafico)
database.database()
database.graficos()

RELATÓRIO DE PREVISÃO DO TEMPO 31-10-2023:

Temperatura:
Máxima 29.0 ºC
Mínima 25.4 ºC

Umidade:
Máxima 29.0 %
Mínima 25.4 %

Pressão:
Máxima 29.0 hPa
Mínima 25.4 hPa

Velocidade do Vento:
Máxima 29.0 m/s
Mínima 25.4 m/s

O tempo será predominantemente de: nuvens dispersas


ValueError: Invalid property specified for object of type plotly.graph_objs.scatter.Line: 'labels'

Did you mean "dash"?

    Valid properties:
        color
            Sets the line color.
        dash
            Sets the dash style of lines. Set to a dash type string
            ("solid", "dot", "dash", "longdash", "dashdot", or
            "longdashdot") or a dash length list in px (eg
            "5px,10px,2px,2px").
        shape
            Determines the line shape. With "spline" the lines are
            drawn using spline interpolation. The other available
            values correspond to step-wise line shapes.
        simplify
            Simplifies lines by removing nearly-collinear points.
            When transitioning lines, it may be desirable to
            disable this so that the number of points along the
            resulting SVG path is unaffected.
        smoothing
            Has an effect only if `shape` is set to "spline" Sets
            the amount of smooth